In [4]:
import pandas as pd
import subprocess
import os

In [7]:
cases_dict = {
    0: {"n": [2], "r": 10, "k": 2},
    1: {"n": [2,4], "r": 10, "k": 2},
    # 2: {"n": [2,4,8,16,32,64], "r": 5, "k": 1},
    # 3: {"n": [2,4,8,16,32,64,128], "r": 3, "k": 0}
}
file_path = r'C:\Users\anabe\Desktop\be\ufrj\sistemas_distribuidos\sistemas_distribuidos_tp3\log.txt'

In [18]:
df_final = pd.DataFrame()
for case in cases_dict:
    try:
        os.remove(file_path)
        print(f"File '{file_path}' deleted successfully.")
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
    except Exception as e:
        print(f"An error occurred while trying to delete the file: {e}")

    with open("log.txt", "w") as file:
        # Write the first line to the file
        file.write("Thread ID, Time, Message Type\n")  # \n inserts a line break

    n, r, k = cases_dict[case]["n"], cases_dict[case]["r"], cases_dict[case]["k"]
    for i in n:
        try:
            process = subprocess.run(["wsl", "./gerenciador_processos", str(i), str(r), str(k)], capture_output=True, text=True, check=True)
            if process.stderr:
                print("C script errors:")
                print(process.stderr)
        except subprocess.CalledProcessError as e:
            print(f"Error running C script: {e}")
        except FileNotFoundError:
            print("C script executable not found.")
    
        df = pd.read_csv(file_path)
        df['n'] = i
        df['r'] = r
        df['k'] = k
        df['case'] = case
        df_final = pd.concat([df_final, df])
    df_final.to_csv(f'case_{case}.csv', index=False)

File 'C:\Users\anabe\Desktop\be\ufrj\sistemas_distribuidos\sistemas_distribuidos_tp3\log.txt' deleted successfully.


In [19]:
df

,Thread ID,Time,Message Type
0,Thread ID: 1,Time: 02:39:01:072 19/07/2023,sent GRANT message
1,Thread ID: 2,Time: 02:39:03:112 19/07/2023,sent GRANT message
2,Thread ID: 1,Time: 02:39:05:130 19/07/2023,sent GRANT message
3,Thread ID: 2,Time: 02:39:07:141 19/07/2023,sent GRANT message
4,Thread ID: 1,Time: 02:39:09:150 19/07/2023,sent GRANT message
5,Thread ID: 2,Time: 02:39:11:162 19/07/2023,sent GRANT message
6,Thread ID: 1,Time: 02:39:13:171 19/07/2023,sent GRANT message
7,Thread ID: 2,Time: 02:39:15:182 19/07/2023,sent GRANT message
8,Thread ID: 1,Time: 02:39:17:193 19/07/2023,sent GRANT message
9,Thread ID: 2,Time: 02:39:19:204 19/07/2023,sent GRANT message
